# CI / CD with CML API V2

##### Cloudera Machine Learning exposes a REST API that you can use to perform operations related to projects, jobs, and runs. You can use API commands to integrate CML with third-party workflow tools or to control CML from the command line.

##### This notebook demonstrates how to create and execute three CML jobs with the CML API V2. 

For the Public Documentation please visit this [page](https://docs.cloudera.com/machine-learning/cloud/api/topics/ml-api-v2.html)

For a full example of the API's capabilities, please visit this notebook from [Cloudera's Public GitHub](https://github.com/cloudera/CML_AMP_APIv2)

#### To use the Python API in your own code, first install the Python API client and point it to your cluster.

In [1]:
### Install the API

import os
cluster = os.getenv("CDSW_DOMAIN")

# If you are not on a TLS enabled cluster (your cluster url starts with ‘http’),
# please use the following command instead.
# !pip3 install http://{cluster}/api/v2/python.tar.gz
#!pip3 install https://{cluster}/api/v2/python.tar.gz

In [2]:
from cmlapi.utils import Cursor
import cmlapi
import string
import random
import json

try:
    client = cmlapi.default_client()
except ValueError:
    print("Could not create a client. If this code is not being run in a CML session, please include the keyword arguments \"url\" and \"cml_api_key\".")

session_id = "".join([random.choice(string.ascii_lowercase) for _ in range(6)])
session_id

'oeqjbk'

##### The API has a lot of cool features. For example, it allows you to view all runtimes associated with the CML Workspace

In [4]:
# cursor also supports search_filter
# cursor = Cursor(client.list_runtimes, 
#                 search_filter = json.dumps({"image_identifier":"jupyter"}))
cursor = Cursor(client.list_runtimes)
runtimes = cursor.items()

In [5]:
from __future__ import print_function
import time
import cmlapi
from cmlapi.rest import ApiException
from pprint import pprint

try:
    # List the available runtime addons, optionally filtered, sorted, and paginated.
    api_response = client.list_runtime_addons(page_size=500)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling CMLServiceApi->list_runtime_addons: %s\n" % e)


{'next_page_token': '',
 'runtime_addons': [{'component': 'HadoopCLI',
                     'display_name': 'Hadoop CLI - CDP 7.2.10 - HOTFIX-1',
                     'identifier': 'hadoop-cli-7.2.10-hf1',
                     'status': 'AVAILABLE'},
                    {'component': 'HadoopCLI',
                     'display_name': 'Hadoop CLI - CDP 7.2.11 - HOTFIX-4',
                     'identifier': 'hadoop-cli-7.2.11-hf4',
                     'status': 'AVAILABLE'},
                    {'component': 'HadoopCLI',
                     'display_name': 'Hadoop CLI - CDP 7.2.14',
                     'identifier': 'hadoop-cli-7.2.14',
                     'status': 'AVAILABLE'},
                    {'component': 'HadoopCLI',
                     'display_name': 'Hadoop CLI - CDP 7.2.8 - HOTFIX-1',
                     'identifier': 'hadoop-cli-7.2.8-hf1',
                     'status': 'AVAILABLE'},
                    {'component': 'Spark',
                     'display_name': 'Spar

##### Similarly, you can see all jobs in the current CML Project

In [6]:
### GET ALL PREVIOUS JOBS FROM PROJECT ###
    
project_id = os.environ["CDSW_PROJECT_ID"]

joblists = client.list_jobs(project_id = project_id)
print(f'Fetched {len(joblists.jobs)} jobs from the project')

Fetched 0 jobs from the project


In [7]:
project_id

'7vvz-ir4l-nvfi-ehzk'

## You can build ML Ops Pipelines by creating and executing CML Jobs with the API from this notebook.

##### First we create a CML Job to train a model. We trained this model in ProtoType.py but in a real world scenario you may have created your model baseline with ML Flow, also available in CML under the Experiments tab.

In [12]:
cml_runtime = "docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-jupyterlab-python3.8-standard:2022.04.1-b6"

In [13]:
### CREATE A JOB TO TEST THE DATA

# Create a dependent job by specifying the parent job's ID in the parent_job_id field.
preprocessing_job_body = cmlapi.CreateJobRequest(
    project_id = project_id,
    name = "1_preprocessing-job",
    script = "cicd/1_preprocessing.py",
    kernel = "python3",
    cpu = 2,
    memory = 4,
    runtime_identifier = cml_runtime
)
preprocessing_job = client.create_job(preprocessing_job_body, project_id)

In [14]:
### CREATE A JOB TO TEST THE DATA

# Create a dependent job by specifying the parent job's ID in the parent_job_id field.
testing_job_body = cmlapi.CreateJobRequest(
    project_id = project_id,
    name = "2_unit-tests-job",
    script = "cicd/2_testing.py",
    kernel = "python3",
    cpu = 1,
    memory = 2,
    runtime_identifier = cml_runtime,
    parent_job_id = preprocessing_job.id
)
testing_job = client.create_job(testing_job_body, project_id)

In [15]:
### CREATE A JOB TO RETRAIN THE MODEL ###
    
# Create a job. We will create dependent/children jobs of this job, so we call this one a "grandparent job". The parameter "runtime_identifier" is needed if this is running in a runtimes project.
train_job_body = cmlapi.CreateJobRequest(
    project_id = project_id,
    name = "3_train-job",
    script = "cicd/3_exp_train_rf.py",
    cpu = 4.0,
    memory = 16.0,
    runtime_identifier = cml_runtime,
    parent_job_id = testing_job.id
)
# Create this job within the project specified by the project_id parameter.
train_job = client.create_job(train_job_body, project_id)

##### A second CML Job to push the model to a REST Endpoint can be set to execute as a dependency on the first one

In [16]:
### CREATE A JOB TO PUSH THE MODEL TO A REST ENDPOINT ###

# Create a dependent job by specifying the parent job's ID in the parent_job_id field.
deploy_job_body = cmlapi.CreateJobRequest(
    project_id = project_id,
    name = "4_deploy-model-job",
    script = "cicd/4_deploy.py",
    kernel = "python3",
    cpu = 2,
    memory = 4,
    runtime_identifier = cml_runtime,
    parent_job_id = train_job.id
)
#    runtime_addon_identifiers = ["spark311-13-hf1"],
deploy_job = client.create_job(deploy_job_body, project_id)

##### A third CML Job to push the model to a REST Endpoint can be set to execute as a dependency on the first one

##### Finally a third job can be created to perform Model Inference. This job is dependent on the second job.

In [18]:
### CREATE A JOB TO DO INFERENCE ON THE MODEL ###

# Create a job that is dependent on the job from the previous cell. This leads to a dependency chain of grandparent_job -> parent_job -> child_job. If grantparent_job runs and succeeds, then parent_job will trigger, and if parent_job runs and succeeds, child_job will trigger. This one uses a template script that does not terminate, so we'll have the opportunity to try stopping it later.
inference_job_body = cmlapi.CreateJobRequest(
    project_id = project_id,
    name = "5_inference-job",
    script = "cicd/5_inference.py",
    kernel = "python3",
    cpu = 2,
    memory = 4,
    runtime_identifier = cml_runtime,
    parent_job_id = deploy_job.id
)
inference = client.create_job(inference_job_body, project_id)

##### Next, we can use the API to run the first job. When it succeeds, the second job will execute, and then the third...

In [ ]:
# Create a job run for the specified job.
# If the job has dependent jobs, the dependent jobs will run after the job succeeds.
# In this case, the grandparent job will run first, then the parent job, and then the child job, provided each job run succeeds.
jobrun_body = cmlapi.CreateJobRunRequest(project_id, preprocessing_job.id)
job_run = client.create_job_run(jobrun_body, project_id, preprocessing_job.id)
run_id = job_run.id